# API Verilerini Alma

In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime, timedelta

# Service Account bilgilerinizi ayarlayın
SERVICE_ACCOUNT_FILE = 'ck-health-indexing-api-dc7562c3e0e0.json'
SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly']

# Site URL'nizi ayarlayın
SITE_URL = "https://ckhealthturkey.com/"

# Başlangıç ve bitiş tarihlerini ayarlayın
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 8, 7)

# API istekleri için maksimum satır sayısı
maxRows = 25000

# Çıktı verilerini saklamak için bir liste oluşturun
output_rows = []

def main():
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('searchconsole', 'v1', credentials=credentials)

    # Tarih aralığını döngüye alın
    for date in date_range(start_date, end_date):
        date_str = date.strftime("%Y-%m-%d")
        print(f"Veriler {date_str} tarihi için çekiliyor...")

        i = 0
        while True:
            request = {
                'startDate': date_str,
                'endDate': date_str,
                'dimensions': ["query", "page", "country", "device"],
                "searchType": "Web",
                'rowLimit': maxRows,
                'startRow': i * maxRows
            }

            response = service.searchanalytics().query(siteUrl=SITE_URL, body=request).execute()

            if 'rows' not in response:
                print(f"{date_str} tarihindeki veriler için yanıt alınamadı. Devam ediliyor...")
                break

            for row in response['rows']:
                keyword = row['keys'][0]
                page = row['keys'][1]
                country = row['keys'][2]
                device = row['keys'][3]
                output_row = [date_str, keyword, page, country, device, row['clicks'], row['impressions'],
                              row['ctr'],
                              row['position']]
                output_rows.append(output_row)

            if len(response['rows']) < maxRows:
                # Son sayfaya ulaşıldı, bir sonraki güne geç
                break
            else:
                i += 1

    # Verileri işleyin (örneğin, CSV dosyasına yazdırın)
    process_data(output_rows)

def date_range(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

def process_data(data):
    import csv
    with open('search_console_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(
            ["Date", "Keyword", "Page", "Country", "Device", "Clicks", "Impressions", "CTR", "Position"])
        writer.writerows(data)  # Ham veriyi doğrudan yazdır


if __name__ == "__main__":
    main()


# average_gun Hesaplama 
#### (URL'nin toplam gösterim aldığı gün sayısının ilk gösterim aldığı tarihten güncel tarihe kadarki süreye oranı: average_gun)

In [ ]:
import pandas as pd
from datetime import datetime

# Veri setini yükle
df = pd.read_csv("search_console_data.csv")

# Tarih sütununu datetime objesine dönüştür
df['Date'] = pd.to_datetime(df['Date'])

# Sonuç DataFrame'i için boş listeler
results = []

# Her URL için hesaplamalar yap
for url in df['Page'].unique():
    # URL'ye ait verileri filtrele
    url_data = df[df['Page'] == url].copy()

    # İlk gösterim tarihini bul
    ilk_gosterim_tarihi = url_data[url_data['Position'] > 0]['Date'].min()

    # İlk gösterim tarihinden 06.08.2024'e kadar geçen gün sayısını hesapla
    toplam_gun = (datetime(2024, 8, 7) - ilk_gosterim_tarihi).days + 1             #BURAYI GÜNCEL TARİHLE DEĞİŞTİRİN 👈👈👈

    # İlk gösterim tarihinden 06.08.2024'e kadar olan verileri filtrele
    url_data = url_data[url_data['Date'] >= ilk_gosterim_tarihi]

    # Gösterim alınan günleri bul (sadece bir kez saymak için unique() kullan)
    gosterim_gunleri = url_data[url_data['Position'] > 0]['Date'].unique()
    gosterim_gun_sayisi = len(gosterim_gunleri)

    # Ortalama gösterim gününü hesapla
    average_gun = gosterim_gun_sayisi / toplam_gun

    # Position değerlerinin ortalamasını hesapla (sadece 0'dan büyük olanlar)
    ortalama_position = url_data[url_data['Position'] > 0]['Position'].mean()

    # Her gün için ortalamadan farkların karesini hesapla
    kare_farklar = [(p - ortalama_position)**2 for p in url_data[url_data['Position'] > 0]['Position']]

    # Kare farkların ortalamasının karekökünü hesapla (standart sapma)
    standart_sapma = (sum(kare_farklar) / len(kare_farklar))**0.5

    # Varyasyon katsayısını hesapla
    degisim_kaysayısı = (standart_sapma / ortalama_position) * 100 if ortalama_position != 0 else 0

    # Sonuçları listeye ekle
    results.append([url, average_gun, gosterim_gun_sayisi, toplam_gun, degisim_kaysayısı])

# Sonuç DataFrame'ini oluştur
results_df = pd.DataFrame(results, columns=['URL', 'average_gun', 'gosterim_gun_sayisi', 'toplam_gun', 'degisim_kaysayısı'])

results_df['average_gun'] = results_df['average_gun'].round(2)

# Sonuçları CSV dosyasına kaydet
results_df.to_csv("sonuclarFINAL2.csv", index=False)

# Gruplama 

In [ ]:
import pandas as pd

def group_and_aggregate_data(csv_file_path):
    """
    Verileri URL'lere göre gruplandırır, aynı URL için aynı kelimeleri birleştirir
    ve tıklama ve gösterim değerlerini toplar.

    Args:
        csv_file_path (str): İşlenecek CSV dosyasının yolu.

    Returns:
        pd.DataFrame: Gruplandırılmış ve toplanmış veriler.
    """

    df = pd.read_csv(csv_file_path, encoding='utf-8')

    grouped_data = df.groupby('Page').agg(
        Keywords=('Keyword', lambda x: '|'.join(set(x))),  # Eşsiz keyword'leri '|' ile birleştir
        TotalClicks=('Clicks', 'sum'),
        TotalImpressions=('Impressions', 'sum')
    ).reset_index()

    return grouped_data

def write_grouped_data_to_csv(grouped_data, output_file_path):
    """
    Gruplandırılmış ve toplanmış verileri CSV dosyasına yazar.

    Args:
        grouped_data (pd.DataFrame): Gruplandırılmış veriler.
        output_file_path (str): Çıkış CSV dosyasının yolu.
    """

    grouped_data.to_csv(output_file_path, index=False, encoding='utf-8')

if __name__ == "__main__":
    csv_file_path = 'search_console_data.csv'  # Giriş CSV dosyasının yolu
    output_file_path = 'grouped_search_console_data.csv'  # Çıkış CSV dosyasının yolu

    grouped_data = group_and_aggregate_data(csv_file_path)
    write_grouped_data_to_csv(grouped_data, output_file_path)
    print(f"Gruplandırılmış veriler '{output_file_path}' dosyasına yazıldı.")

# Kümeler arası benzerlik hesaplama

In [ ]:
import pandas as pd
import re

df = pd.read_csv("grouped_search_console_data.csv")

def create_keyword_sets(row):
  """Her satırdaki 'Keywords' sütunundaki kelimeleri | ile bölerek bir küme oluşturur.
     Regex ile belirtilen kelimeleri hariç tutar."""
  keywords = row['Keywords'].split('|')
  # Regex ile belirtilen kelimeleri hariç tutar
  excluded_pattern = r"^ck.*|\bck\b|ckhealth"  
  keywords = [word for word in keywords if not re.search(excluded_pattern, word)]
  
  return set(keywords)

# Her satırdaki 'Keywords' sütunundaki kelimeleri bir kümeye dönüştürür
df['Keyword Sets'] = df.apply(create_keyword_sets, axis=1)

# Tüm Page değerlerini bir listeye atar
pages = df['Page'].tolist()

# Her sayfanın kelime kümesini karşılaştırır ve benzerlik yüzdesini hesaplar
similarity_results = []

for i in range(len(pages)):
  for j in range(i+1, len(pages)):
    page1_keywords = df['Keyword Sets'][i]
    page2_keywords = df['Keyword Sets'][j]

    common_keywords = page1_keywords.intersection(page2_keywords)
    #  len(page1_keywords) değeri 0 ise similarity'i 0 olarak ayarlar
    similarity = 0 if len(page1_keywords) == 0 else len(common_keywords) / len(page1_keywords) * 100

    # Benzerlik yüzdesini ondalıklı olarak yazdırır
    similarity_results.append({
        'Page 1': pages[i],
        'Page 2': pages[j],
        'Similarity (%)': int(similarity)
    })

# Sonuçları bir DataFrame'e dönüştürür
similarity_df = pd.DataFrame(similarity_results)

# DataFrame'i yazdırır
print(similarity_df)

similarity_df.to_csv("kümelemeFINAL.csv")